In [17]:
import pandas as pd
from funciones import *

In [18]:
file_path_steam_games = '../data/raw/output_steam_games.json'

In [19]:
df_games = process_file(file_path_steam_games, '../data/interim/csv/games.csv', load_and_normalize_steam_games)

In [20]:
df_games.dropna(how='all', inplace=True)
df_games.reset_index(drop=True, inplace=True)

for column in df_games.columns:
    if column not in ['early_access', 'id', 'release_date', 'price']:
        df_games[column] = df_games[column].astype(str)

# Manejando las columnas que requieren un tratamiento especial
df_games['early_access'] = df_games['early_access'].astype(bool)
df_games['id'] = df_games['id'].astype('Int64')
df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')

# Copiar la columna 'price' a 'price_converted'
df_games['price_converted'] = df_games['price']
df_games['price_converted'] = df_games['price_converted'].apply(convert_price)

df_games.drop(columns=['price', 'title', 'url', 'reviews_url', 'publisher', 'genres', 'specs', 'early_access'], inplace=True)

df_games.rename(columns={'price_converted': 'price'}, inplace=True)
df_games['id'] = df_games['id'].fillna(-1).astype(int)
df_games['price'] = df_games['price'].fillna(-1).astype(float)


In [21]:

# Reemplazar valores nulos con 'Valor Desconocido (NULL)'
df_games[['app_name']] = df_games[['app_name']].fillna('Valor Desconocido (NULL)')
df_games[['developer']] = df_games[['developer']].fillna('Valor Desconocido (NULL)')

# Verificar que no hay más valores nulos en el DataFrame
# print(df_games_clean.isnull().sum())

# Aplicar la función a las columnas 'genres', 'tags', y 'specs'
for column in [ 'tags']:
    df_games[column] = df_games[column].apply(convert_string_to_list)

# # Verificar los cambios
# print(df_games[['genres', 'tags', 'specs', 'release_date']].head())

df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')

# Extract the year from the release_date column
df_games['release_year'] = df_games['release_date'].dt.year

# Fill NaT values with a placeholder value, here we use -1
df_games['release_year'] = df_games['release_year'].fillna(-1).astype(int)  # Combined fillna and astype

# Now, drop the release_date column as it's no longer needed
df_games.drop(columns=['release_date'], inplace=True)

In [22]:
df_games = pd.read_parquet('../data/interim/parquet/clean_games.parquet')

In [23]:
from sklearn.preprocessing import MultiLabelBinarizer

# Instanciar MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Ajustar y transformar los datos
one_hot_encoded = mlb.fit_transform(df_games['tags'])

# Crear un DataFrame a partir de la codificación one-hot
df_one_hot_encoded = pd.DataFrame(one_hot_encoded, columns=mlb.classes_)

# Concatenar el DataFrame original con el DataFrame one-hot encoded
df_games = pd.concat([df_games, df_one_hot_encoded], axis=1)

# Opcional: eliminar la columna 'tags' original
df_games.drop(columns='tags', inplace=True)

# Mostrar las primeras filas del DataFrame
print(df_games.head())


                  app_name      id         developer  price  release_year  \
0      Lost Summoner Kitty  761140         Kotoshiro   4.99          2018   
1                Ironbound  643980  Secret Level SRL   0.00          2018   
2  Real Pool 3D - Poolians  670290      Poolians.com   0.00          2017   
3                  弹炸人2222  767400              彼岸领域   0.99          2017   
4            Log Challenge  773570               nan   2.99            -1   

   1980s  1990's  2.5D  2D  2D Fighter  ...  Warhammer 40K  Web Publishing  \
0      0       0     0   0           0  ...              0               0   
1      0       0     0   1           0  ...              0               0   
2      0       0     0   0           0  ...              0               0   
3      0       0     0   0           0  ...              0               0   
4      0       0     0   0           0  ...              0               0   

   Werewolves  Western  Word Game  World War I  World War II  Wrestl

# Reduccion de dimensionalidad

In [24]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Columns: 344 entries, app_name to e-sports
dtypes: float64(1), int32(341), object(2)
memory usage: 42.5+ MB


In [25]:
# Obtener el índice de la columna 'release_year'
tag_columns_start_idx = df_games.columns.get_loc('release_year') + 1

# Ahora sumar solo las columnas de tags
tag_counts = df_games.iloc[:, tag_columns_start_idx:].sum()

# Calcular el porcentaje de cada tag
tag_percentage = (tag_counts / len(df_games)) * 100

# Ordenar los tags por su porcentaje de ocurrencia (de mayor a menor)
sorted_tag_percentage = tag_percentage.sort_values(ascending=False)

# Imprimir los porcentajes por columna antes de filtrar
print("Porcentajes por columna antes de filtrar:\n", sorted_tag_percentage)

# Definir el umbral (puedes cambiar este valor para experimentar)
umbral = 3

# Seleccionar tags basados en el umbral definido
significant_tags = sorted_tag_percentage[sorted_tag_percentage > umbral]

# Mantener solo las columnas de tags significativas junto con las primeras columnas originales
columns_to_keep = list(df_games.columns[:tag_columns_start_idx]) + list(significant_tags.index)
df_games = df_games[columns_to_keep]

# Sumar solo las columnas de tags significativas en el DataFrame actualizado
significant_tag_counts = df_games.iloc[:, tag_columns_start_idx:].sum()

# Calcular el porcentaje de cada tag significativo
significant_tag_percentage = (significant_tag_counts / len(df_games)) * 100

# Ordenar los tags significativos por su porcentaje de ocurrencia (de mayor a menor)
sorted_significant_tag_percentage = significant_tag_percentage.sort_values(ascending=False)

# Imprimir los porcentajes por columna después de filtrar
print("\nPorcentajes por columna después de filtrar:\n", sorted_significant_tag_percentage)


Porcentajes por columna antes de filtrar:
 Indie            54.831181
Action           40.469893
Adventure        30.592812
Casual           30.306519
Simulation       24.054769
                   ...    
Underground       0.018671
Voice Control     0.015559
Cycling           0.015559
Faith             0.012447
Bikes             0.006224
Length: 339, dtype: float64

Porcentajes por columna después de filtrar:
 Indie                    54.831181
Action                   40.469893
Adventure                30.592812
Casual                   30.306519
Simulation               24.054769
Strategy                 23.544422
RPG                      18.419169
Singleplayer             13.524195
Free to Play              7.487164
Multiplayer               7.415590
Great Soundtrack          6.955033
Puzzle                    6.569161
Early Access              6.058814
2D                        6.052591
Atmospheric               6.021472
VR                        5.588922
Sports                    

In [26]:
# Guardar el DataFrame actualizado
df_games.to_csv('../data/interim/csv/final_games.csv', index=False)
df_games.to_parquet('../data/interim/parquet/final_games.parquet')


In [27]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   app_name               32135 non-null  object 
 1   id                     32135 non-null  int32  
 2   developer              32135 non-null  object 
 3   price                  32135 non-null  float64
 4   release_year           32135 non-null  int32  
 5   Indie                  32135 non-null  int32  
 6   Action                 32135 non-null  int32  
 7   Adventure              32135 non-null  int32  
 8   Casual                 32135 non-null  int32  
 9   Simulation             32135 non-null  int32  
 10  Strategy               32135 non-null  int32  
 11  RPG                    32135 non-null  int32  
 12  Singleplayer           32135 non-null  int32  
 13  Free to Play           32135 non-null  int32  
 14  Multiplayer            32135 non-null  int32  
 15  Gr

In [28]:
print(df_games.head())

                  app_name      id         developer  price  release_year  \
0      Lost Summoner Kitty  761140         Kotoshiro   4.99          2018   
1                Ironbound  643980  Secret Level SRL   0.00          2018   
2  Real Pool 3D - Poolians  670290      Poolians.com   0.00          2017   
3                  弹炸人2222  767400              彼岸领域   0.99          2017   
4            Log Challenge  773570               nan   2.99            -1   

   Indie  Action  Adventure  Casual  Simulation  ...  Pixel Graphics  Co-op  \
0      1       1          0       1           1  ...               0      0   
1      1       0          0       0           0  ...               0      0   
2      1       0          0       1           1  ...               0      0   
3      0       1          1       1           0  ...               0      0   
4      1       1          0       1           0  ...               0      0   

   Shooter  Racing  Female Protagonist  Funny  First-Person  F